In [ ]:
# default_exp core

# Calorie counter

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

First you will provide your height, weight and age and calculate your Basal Metabolic Rate, the number of calories required to keep your body functioning at rest.

After choosing your usual level of activity the Active Metabolic Rate, the number of calories that we consume on a daily basis depending on our height, gender, age, weight and entered activity level whilst maintaining current weight, will be calculated.

Than you will be asked to type in the activity you were doing today. If there are more than one activities matching your input, you will select one from the lst that matches your search pattern. Additionally you will enter the time you were doing this activity.

source for the formulas:    https://www.verywellfit.com/how-many-calories-do-i-need-each-day-2506873
source for the MET values:  https://golf.procon.org/met-values-for-800-activities/

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
import pandas as pd
import regex as re

In [ ]:
#export
def input_body_parameters(message_dict):
    """
    returns a dictionary with user input of body parameters: height, weight, age, gender and activity level

    Parameters: message_dict: dict
                a dictionary with body parameters as keys and message for user as values

    Raises: AssertError: if inputs of type int are negative
            AssertError: if inputs of type string are not 'w' or 'm'
    
    Return: dict
            a dictionary with body parameters as keys and their values as values
    """
    input_values = [170, 56, 41,'m', 1]
    body_params = {}
    n = range(len(message_dict))
    zipped = zip(message_dict.keys(), n)
    assert len(input_values) == len(message_dict)
    for param, nr in zipped:
        if param =='gender':
            try:
                body_params[param] = input(message_dict[param])
            except:
                body_params[param] = input_values[nr]
            assert body_params[param] == 'w' or body_params[param] == 'm', 'gender value should be w or m'
        else:
            try:
                body_params[param] = int(input(message_dict[param]))
            except:
                body_params[param] = input_values[nr]
            assert body_params[param] > 0, 'Please provide a positive value'

        body_params[param] = body_params[param]

        print(f'Your {param} is {body_params[param]}')
    return body_params


def calculate_bmr_amr(body_parameters):
    """
    returns basal metabolic rate (bmr) and active metabolic rate (amr) 
    
    Parameters: body_parameters: dict 
                dictionary with body parameters as keys and their values as values

    Returns: float
            basal metabolic rate (bmr)
            float
            active metabolic rate (amr)
    """
    dict_activity = {1:1.2, 2:1.37, 3:1.55, 4:1.725, 6:1.9}

    if body_parameters['gender'] == 'w':
        bmr = float(655.1 + (9.563 * body_parameters['weight']) +
            (1.850 * body_parameters['height']) - 
            (4.676 * body_parameters['age']))
    else:
        bmr = float(66.47 + (13.75 * body_parameters['weight']) + 
            (5.003 * body_parameters['height']) - 
            (6.755 * body_parameters['age']))

    amr = bmr * dict_activity[body_parameters['activity']]
    return bmr, amr
    

def calculate_burned_calories(body_parameters):
    """
    Returns the amount of calories burned during activity and during the day apart of activity.
    
    To calculate calories for specific activity its MET value is chosen from the table.
    To assign MET user should type in the name of her activity for the search.
    If user doesn't provide this information, activity is considered to be standing and light effort with MET of 1.4
    Calories burned during rest of the day are calculated with standard MET of 1.2.
    
    Parameters: body_parameters: dict
                dictionary with body parameters as keys and their values as values

    Returns: float
            amount of calories burned during activity
            float
            amount of calories burned during the day apart of activity
    """
    import pandas as pd
    
    met_list = pd.read_csv('data/met_list_activities.csv',
                            sep=';', 
                            #encoding='ANSI', 
                            decimal=',', 
                            error_bad_lines=False)
    nr = 0
    met = 0 
    time = 0
    your_act = ''
    while met == 0:
        try:
            your_act = input('Type in the activity you were doing today:')
        except:
            your_act = 'standing, light effort tasks'
        # cheking, if the typed activity is in the list of activities
        if len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) > 1:
            print('\n', met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)])
            try:
                nr = int(input('\nSelect MET of your activity from the list'))
            except:
                nr = 176
        elif len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) == 0:
            print('Please type again')
        elif len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) == 1:
            nr = met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)].index
        met = float(met_list['METs'].iloc[nr])
        activity = str(met_list['SPECIFIC MOTION'].iloc[nr])
    try:
        time = int(input('Type in the time of your activity for today in minutes:'))
    except:
        time = 30

    burned_cal = met * body_parameters['weight'] * time/60
    bmr_cal = 1.2 * body_parameters['weight'] * (24 - time / 60)
    print(f'\nDuring {time} min of {activity} you burned {burned_cal:.0f} kcal.')
    print(f'Your total daily energy expenditure was {round((burned_cal + bmr_cal), 2):.0f} kcal today')
    return burned_cal, bmr_cal

In [ ]:
#hide
# idea! add possibility to correct initial data

message_activity_level = 'Please choose your level of activity:\
                \n1 - little to no exercise \
                \n2 - light exercise 1–3 days a week \
                \n3 - moderate exercise 3–5 days a week \
                \n4 - hard exercises 6–7 days a week \
                \n5 - physically demanding job or particularly challenging exercise routine'

message_dict = {'height': 'Input height in cm',
                'weight': 'Input weight in kg',
                'age': 'Input age in full years',
                'gender': 'Input biological gender as w or m',
                'activity': message_activity_level}

body_params = input_body_parameters(message_dict)

bmr, amr = calculate_bmr_amr(body_params)
print(f'\nYour BMR is {bmr:.1f}')
print(f'To stay at your current weight you need to consume {amr:.0f} calories')

Your height is 170
Your weight is 55
Your age is 25
Your gender is w
Your activity is 1

Your BMR is 1378.7
To stay at your current weight you need to consume 1654 calories


In [ ]:
#hide
print('Please provide your weight, height and age in the first part of the calculator')

burned_cal, bmr_cal = calculate_burned_calories(body_params)

C:\Users\AFEDOR~1\AppData\Local\Temp/ipykernel_34744/2494668240.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  burned_cal, bmr_cal = calculate_burned_calories(body_params)


Please provide your weight, height and age in the first part of the calculator

During 30 min of 174    standing, light effort tasks (pump gas, change...
Name: SPECIFIC MOTION, dtype: object you burned 55 kcal.
Your total daily energy expenditure was 1606 kcal today


In [ ]:
"""
met_list = pd.read_csv('data/met_list_activities.csv',
                            sep=';', 
                            #encoding='ANSI', 
                            decimal=',', 
                            error_bad_lines=False)

print(str(met_list['SPECIFIC MOTION'].iloc[78]))
"""

aerobic, step, with 4-inch step
